# Pyser 2018 - Introduction to neural networks with Keras and Tensorflow
# Welcome to TensorFlow!

TensorFlow™ is an open source software library for high performance numerical computation. Its flexible architecture allows easy deployment of computation across a variety of platforms (CPUs, GPUs, TPUs), and from desktops to clusters of servers to mobile and edge devices. Originally developed by researchers and engineers from the Google Brain team within Google’s AI organization, it comes with strong support for machine learning and deep learning and the flexible numerical computation core is used across many other scientific domains.

https://www.tensorflow.org/

# Check GPU

In [0]:
import tensorflow as tf
from tensorflow.python.client import device_lib

def get_available_gpus():
  local_device_protos = device_lib.list_local_devices()
  return [x.name for x in local_device_protos if x.device_type == 'GPU']

name = get_available_gpus()
print("GPUS:{}".format(name))

!/opt/bin/nvidia-smi

# Basic Tensorflow session

https://www.tensorflow.org/programmers_guide/graphs



In [0]:
import tensorflow as tf
import numpy as np

sess = tf.Session()
print ("TF Session opened")

# Tensorflow constants

https://www.tensorflow.org/api_guides/python/constant_op

In [0]:
def print_tf(x):
    print("type:\n %s" % (type(x)))
    print("value:\n %s" % (x))
hello = tf.constant("Hello pyser!")
print_tf(hello)

In [0]:
# To actually compute:

hello_out = sess.run(hello)
print_tf(hello_out)

# Number constants

In [0]:
a = tf.constant(1.5)
b = tf.constant(2.5)
print_tf(a)
print_tf(b)

a_out = sess.run(a)
b_out = sess.run(b)
print_tf(a_out)
print_tf(b_out)

# Operators

https://www.tensorflow.org/api_guides/python/math_ops

In [0]:
a_plus_b = tf.add(a, b)
print_tf(a_plus_b)

a_plus_b_out = sess.run(a_plus_b)
print_tf(a_plus_b_out)

a_mul_b = tf.multiply(a, b)
a_mul_b_out = sess.run(a_mul_b)
print_tf(a_mul_b_out)

# Variables

https://www.tensorflow.org/programmers_guide/variables

In [0]:
weight = tf.Variable(tf.random_normal([5, 2], stddev=0.1))
print_tf(weight)

init = tf.global_variables_initializer()
sess.run(init)

weight_out = sess.run(weight)
print_tf(weight_out)

# Placeholders

https://stackoverflow.com/questions/36693740/whats-the-difference-between-tf-placeholder-and-tf-variable

In [0]:
x = tf.placeholder(tf.float32, [None, 5])
print_tf(x)

# Complex operations



In [0]:
oper = tf.matmul(x, weight)
print_tf(oper)

data = np.random.rand(1, 5)
oper_out = sess.run(oper, feed_dict={x: data})
print_tf(oper_out)

data = np.random.rand(2, 5)
oper_out = sess.run(oper, feed_dict={x: data})
print_tf(oper_out)

# Pure TensorFlow Neural Net (XOR)

In [0]:
sess = tf.InteractiveSession()

# Desired input output mapping of XOR function:
x_ = [[0, 0], [0, 1], [1, 0], [1, 1]] # input
#labels=[0,      1,      1,      0]   # output =>
expect=[[1,0],  [0,1],  [0,1], [1,0]] # ONE HOT REPRESENTATION! 'class' [1,0]==0 [0,1]==1

x = tf.placeholder("float", [None,2])
y_ = tf.placeholder("float", [None, 2])

number_hidden_nodes = 20 # 20 outputs to create some room for negatives and positives

W = tf.Variable(tf.random_uniform([2, number_hidden_nodes], -.01, .01))
b = tf.Variable(tf.random_uniform([number_hidden_nodes], -.01, .01))
hidden  = tf.nn.relu(tf.matmul(x,W) + b) # first layer.

 # the XOR function is the first nontrivial function, for which a two layer network is needed.
W2 = tf.Variable(tf.random_uniform([number_hidden_nodes,2], -.1, .1))
b2 = tf.Variable(tf.zeros([2]))
hidden2 = tf.matmul(hidden, W2)#+b2

y = tf.nn.softmax(hidden2)

# Define loss and optimizer
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(cross_entropy)

# Train
tf.initialize_all_variables().run()
for step in range(1000):
    feed_dict = {x: x_, y_:expect } # feed the net with our inputs and desired outputs.
    e, a = sess.run([cross_entropy,train_step], feed_dict)
    if e<1:
      break # early stopping
    print ("step %d : entropy %s" % (step,e)) # error/loss should decrease over time

# Test trained model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1)) # argmax along dim-1
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) # [True, False, True, True] -> [1,0,1,1] -> 0.75.

print ("accuracy %s" % (accuracy.eval({x: x_, y_: expect})))

learned_output = tf.argmax(y, 1)
print ("outputs", learned_output.eval({x: x_}))

sess.close()